## p-value with sample size

In [1]:
# !pip install scipy seaborn pingouin -q

In [24]:
import pandas as pd
import numpy as np


In [25]:
# create a dummy data
dft = pd.DataFrame({
    'color' : ['black', 'white'] * 25,
    'r' : np.random.uniform(1.5, 10.2, 50),
    'g' : np.random.uniform(1.5, 14.9, 50),
    'b' : np.random.uniform(4.5, 21.4, 50),
    'brightness' : np.random.randint(1, 100, 50),
})

# np.set_printoptions(precision=2, suppress=True)
dft.head()

,color,r,g,b,brightness
0,black,5.590498,9.368406,12.390344,12
1,white,1.527751,14.778793,19.972003,22
2,black,9.649591,11.379309,12.945859,90
3,white,9.063293,10.780019,15.114241,46
4,black,9.992498,4.807507,19.257925,27


In [26]:
dft.to_csv('color.csv', index=False)

#### t-test

In [27]:
from scipy import stats

df = pd.read_csv('color.csv')
black = pd.DataFrame(df.loc[df['color'] == 'black']['brightness'])
white = pd.DataFrame(df.loc[df['color'] == 'white']['brightness'])

print('Two-sided:', stats.ttest_ind(black, white, equal_var = True))

print('One-sided, black first:', stats.ttest_ind(black, white, equal_var = True, alternative = 'greater'))
print('One-sided, white first:', stats.ttest_ind(white, black, equal_var = True, alternative = 'greater'))

Two-sided: TtestResult(statistic=array([-0.4]), pvalue=array([0.69]), df=array([48.]))
One-sided, black first: TtestResult(statistic=array([-0.4]), pvalue=array([0.66]), df=array([48.]))
One-sided, white first: TtestResult(statistic=array([0.4]), pvalue=array([0.34]), df=array([48.]))


In [28]:
# with diff sample size
from scipy import stats

df = pd.read_csv('color.csv', nrows=20)
black = pd.DataFrame(df.loc[df['color'] == 'black']['brightness'])
white = pd.DataFrame(df.loc[df['color'] == 'white']['brightness'])

print('result:', stats.ttest_ind(black, white, equal_var = True))

result: TtestResult(statistic=array([-0.33]), pvalue=array([0.75]), df=array([18.]))


### regression model

In [29]:
import pingouin as pg

# predictor r
# outcome brightness
mod1 = pg.linear_regression(df['r'], df['brightness'])
# Display results, rounded to two decimal places.
mod1.round(2)

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,62.88,16.39,3.84,0.00,0.02,-0.03,28.44,97.32
1,r,-1.77,2.61,-0.68,0.51,0.02,-0.03,-7.26,3.71


In [30]:
# predictor r, g
# outcome brightness
mod2 = pg.linear_regression(df[['r', 'g']], df['brightness'])
# Display results, rounded to two decimal places.
mod2.round(2)

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,74.27,24.47,3.04,0.01,0.05,-0.06,22.64,125.90
1,r,-2.16,2.72,-0.79,0.44,0.05,-0.06,-7.90,3.59
2,g,-1.10,1.73,-0.64,0.53,0.05,-0.06,-4.76,2.56


In [31]:
# predictor r, g, b
# outcome brightness
mod3 = pg.linear_regression(df[['r', 'g', 'b']], df['brightness'])
# Display results, rounded to two decimal places.
mod3.round(2)

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,85.34,34.62,2.46,0.03,0.06,-0.12,11.94,158.73
1,r,-2.26,2.80,-0.81,0.43,0.06,-0.12,-8.18,3.67
2,g,-0.86,1.85,-0.46,0.65,0.06,-0.12,-4.79,3.07
3,b,-0.97,2.10,-0.46,0.65,0.06,-0.12,-5.41,3.47


- calculate regression co-efficient

In [32]:
# calculate standadize regression co-efficient
from scipy import stats

df['rs'] = stats.zscore(df['r'])
df['gs'] = stats.zscore(df['g'])
df['bs'] = stats.zscore(df['b'])
df['brightness'] = stats.zscore(df['brightness'])

predictors = df[['rs', 'gs', 'bs']]
outcome = df['brightness']

mod3 = pg.linear_regression(predictors, outcome)
mod3.round(4)

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,0.0000,0.2424,0.0000,1.0000,0.0602,-0.116,-0.5138,0.5138
1,rs,-0.2011,0.2492,-0.8067,0.4317,0.0602,-0.116,-0.7294,0.3273
2,gs,-0.1200,0.2592,-0.4630,0.6496,0.0602,-0.116,-0.6696,0.4295
3,bs,-0.1183,0.2554,-0.4632,0.6495,0.0602,-0.116,-0.6596,0.4231


#### model statics 
- we will build an OLS (ordinary least squares) model using the statsmodels library

In [33]:
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

# fit our regression model using statsmodels
fit = smf.ols('brightness ~ r+g+b', data=df).fit()

bptest=sms.diagnostic.het_breuschpagan(fit.resid, fit.model.exog)

print('Lagrange multiplier statistic:', bptest[0])
print('p:', bptest[1])

Lagrange multiplier statistic: 4.144592870155053
p: 0.24626806564682857


In [34]:
# model summary
smf.ols('brightness ~ r+g+b', data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             brightness   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                 -0.116
Method:                 Least Squares   F-statistic:                    0.3418
Date:                Thu, 04 Apr 2024   Prob (F-statistic):              0.795
Time:                        17:58:15   Log-Likelihood:                -27.758
No. Observations:                  20   AIC:                             63.52
Df Residuals:                      16   BIC:                             67.50
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1162      1.186      0.941      0.361      -1.398       3.631
r             -0.0773      0.096     -0.807      0.432      -0.280       0.126
g             -0.0294      0.063     -0.463      0.650      -0.164       0.105
b             -0.0332      0.072     -0.463      0.649      -0.185       0.119
==============================================================================
Omnibus:                        3.614   Durbin-Watson:                   1.605
Prob(Omnibus):                  0.164   Jarque-Bera (JB):                1.471
Skew:                           0.234   Prob(JB):                        0.479
Kurtosis:                       1.757   Cond. No.                         82.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""